In [1]:
import numpy as np
import operator
from math import sqrt
import sklearn
from sklearn import datasets
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import warnings
from sklearn.exceptions import ConvergenceWarning
"""
Property A.1: Algorithm converges to the maximum price vector over time
Description of Experiment 1.1
Take n example datasets from Scikit-Learn [(X_i), Y_i]
[(X_1), (X_2), …, (X_n)] are features on sale
[Y_1, Y_2, …., Y_n] are prediction tasks “for hire”
[b_1, b_2, ….., b_n] are bids for each prediction type  
Dynamics
Uniformly at random select (Y_n, b_n) as stream of buyers
Desired outcomes of experiment  
(P_i) converge to approximately b_i 
Why? Optimal Outcome is that price of each (feature set type) converges to bid for that that prediction task type
Regularization helps
Why? - Theory shows that it makes the problem “convex”
"""
dset_names = [
 'load_boston',
 'load_breast_cancer',
 'load_diabetes',
 'load_digits',
]
    
dsets = {}
for attr in dset_names:
    name = attr.split('load_')[1]
    if name:
        dsets[name] = getattr(datasets, attr)()
Y_dict = { name: dsets[name]['target'] for name in dsets }
X_dict = { name: dsets[name]['data'] for name in dsets }

In [2]:
[(key, x.shape) for key, x in X_dict.items()]

[('boston', (506, 13)),
 ('breast_cancer', (569, 30)),
 ('diabetes', (442, 10)),
 ('digits', (1797, 64))]

In [3]:
[(key, y.shape) for key, y in Y_dict.items()]

[('boston', (506,)),
 ('breast_cancer', (569,)),
 ('diabetes', (442,)),
 ('digits', (1797,))]

In [4]:
""" Look at first four, up to 400 time steps"""
X_t1 = np.concatenate([x[:400] for x in X_dict.values()], axis=1)
Y_t1 = np.stack([y[:400] for y in Y_dict.values()])

In [5]:
X_t1.shape, Y_t1.shape

((400, 117), (4, 400))

In [6]:

def get_original_columns(X_n, feature_idxs):
    indices = [idx for (idx, price) in feature_idxs]
    return X_n[:, indices]

def gain_score(y, x):
#     print('y: {}'.format(y))
#     print('x: {}'.format(x))
    r2 = r2_score(y, x)
#     print('r2: {}'.format(r2))
    return r2 if r2 > 0 else 0

In [28]:
# https://arxiv.org/pdf/1805.08125.pdf
# Allocation and Revenue function
# Section 4.1
def revenue_func(P_n, b_n, Y_n, context, random_state):
    """
    P_n: the price vector at previous timestep (t=n)
    b_n: the bid at previous timestep (in $ / model_score)
    Y_n: the test Y data set
    """
    X_n = context.get('X_n')
    gain_func = context.get('gain_func')
    if X_n is None or gain_func is None:
        return None
    
    feature_count = len(P_n)
    feature_options = sorted(list((idx, p) for (idx, p) in enumerate(P_n) if p <= b_n), key=operator.itemgetter(1))
    print('feature_options: {}'.format(feature_options))
    if not feature_options:
        return 0
    
    assert feature_count == X_n.shape[1], 'feature_count: {}, X_n.shape[1]: {}'.format(feature_count, X_n.shape[1])

    # 90% train and cross-validate, 10% test
    print('X_n shape: {}'.format(X_n.shape))
    print('Y_n shape: {}'.format(Y_n.shape))
    X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, Y_n, test_size=0.1, random_state=random_state)

    # Gain results by slice index
    feature_indices = []
    gain_results = []
    for (idx, (feature_idx, feature_price)) in enumerate(feature_options):
        _train = get_original_columns(X_n_train, feature_idxs=feature_options[:idx+1])
        _test = get_original_columns(X_n_test, feature_idxs=feature_options[:idx+1])
        gain_func.fit(_train, y_n_train)
        result = gain_score(y_n_test, gain_func.predict(_test))
        previous_result = gain_results[-1] if gain_results else 0
        if (result > previous_result):
            gain_results.append(result)
            feature_indices.append(feature_idx)

    if not gain_results:
        raise Exception("gain_results empty")

    positive_amount = b_n * gain_results[-1]
    negative_amount = 0
    # Add the bid as the rightmost price
    feature_indices += [-1]
    P_n += [b_n]
    for idx, gain in enumerate(gain_results):
        price_idx = feature_indices[idx]
        price_idx_plus_one = feature_indices[idx+1]
        price_difference = P_n[price_idx_plus_one] - P_n[price_idx]
        assert(price_difference >= 0)
        negative_amount += price_difference * gain
    return positive_amount - negative_amount


In [29]:
X_n = X_t1
X_n.shape

(400, 117)

In [32]:
# Test 1: price of all features equal to bid (all allocated, revenue = best gain * bid)
def test1(random_state=100):
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'gain_func': clf,
        'X_n': X_n,
    }
    feature_count = X_n.shape[1]
    b_n = 5
    P_n = [b_n] * feature_count
    
    # TODO: test each instead of randomly
    y_choice_idx = np.random.choice(Y_t1.shape[0])
    Y_n = Y_t1[y_choice_idx]

    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    revenue = revenue_func(P_n, b_n, Y_n, context, random_state)

    X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n, Y_n, test_size=0.1, random_state=random_state)
    clf.fit(X_n_train, y_n_train)
    expected_revenue = gain_score(y_n_test, clf.predict(X_n_test)) * b_n
    print(expected_revenue)
#     assert np.isclose(revenue, expected_revenue), "{} != {}".format(revenue, expected_revenue)
    assert revenue >= expected_revenue, "{} != {}".format(revenue, expected_revenue)
test1()

feature_options: [(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 5), (17, 5), (18, 5), (19, 5), (20, 5), (21, 5), (22, 5), (23, 5), (24, 5), (25, 5), (26, 5), (27, 5), (28, 5), (29, 5), (30, 5), (31, 5), (32, 5), (33, 5), (34, 5), (35, 5), (36, 5), (37, 5), (38, 5), (39, 5), (40, 5), (41, 5), (42, 5), (43, 5), (44, 5), (45, 5), (46, 5), (47, 5), (48, 5), (49, 5), (50, 5), (51, 5), (52, 5), (53, 5), (54, 5), (55, 5), (56, 5), (57, 5), (58, 5), (59, 5), (60, 5), (61, 5), (62, 5), (63, 5), (64, 5), (65, 5), (66, 5), (67, 5), (68, 5), (69, 5), (70, 5), (71, 5), (72, 5), (73, 5), (74, 5), (75, 5), (76, 5), (77, 5), (78, 5), (79, 5), (80, 5), (81, 5), (82, 5), (83, 5), (84, 5), (85, 5), (86, 5), (87, 5), (88, 5), (89, 5), (90, 5), (91, 5), (92, 5), (93, 5), (94, 5), (95, 5), (96, 5), (97, 5), (98, 5), (99, 5), (100, 5), (101, 5), (102, 5), (103, 5), (104, 5), (105, 5), (106, 5), (107, 5), (108, 5), (1

In [33]:
def test2(random_state=100):
    # Test that no features are allocated (revenue zero) when all price are greater than the bid
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'gain_func': clf,
        'X_n': X_n,
    }
    feature_count = X_n.shape[1]
    b_n = 0.05
    P_n = [0.1] * feature_count
    y_choice_idx = np.random.choice(Y_t1.shape[0])
    Y_n = Y_t1[y_choice_idx]
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    revenue = revenue_func(P_n, b_n, Y_n, context, random_state)
    assert revenue == 0
test2()

feature_options: []


In [35]:
def test3(feature_count=3, random_state=100):
    clf = linear_model.LassoCV(cv=5, random_state=random_state)
    context = {
        'gain_func': clf,
        'X_n': X_n[:, :feature_count],
    }
    b_n = 5
    P_n = [6, 3, 2]
    Y_n = Y_t1[3]
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    revenue = revenue_func(P_n, b_n, Y_n, context, random_state)
    print(revenue)
    P_n = [1, 1, 6]
    revenue = revenue_func(P_n, b_n, Y_n, context, random_state)
    print(revenue)
test3()

feature_options: [(2, 2), (1, 3)]
X_n shape: (400, 3)
Y_n shape: (400,)


Exception: gain_results empty